In [1]:
from sklearn.metrics import mean_squared_error
from skimage.metrics import structural_similarity as ssim
# from skimage import data, img_as_float
import pandas as pd
import cv2
import lpips
import os
import torch
pd.options.mode.chained_assignment = None

In [2]:
import scipy.io as sio
from scipy import stats
test = sio.loadmat('hw5/hw5.mat')

In [3]:
## Preprocessing the mat file to generate a dataframe without the meta information
df = pd.DataFrame(test.items())
df = df.T
df = df.drop(df.columns[[0, 1, 2]], 1)
df = df.drop([0], 0)
df.head()

,3,4,5
1,"[[42.127765361449555, 37.462720322546936, 43.2...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[[caps.bmp], [churchandcapitol.bmp], [monarch..."


In [4]:
datas = list(zip(df[5][1][0][:].reshape(-1),df[3][1][:].reshape(-1),df[4][1][:].reshape(-1)))
dfnew = pd.DataFrame(datas, columns = ['refnames_blur','blur_dmos', 'blur_orgs'])
dfnew.head()

## Drop the rows with original images
dfnew.drop(dfnew[dfnew['blur_orgs'] == 1].index, inplace = True)
dfnew.head()

,refnames_blur,blur_dmos,blur_orgs
0,[caps.bmp],42.127765,0
1,[churchandcapitol.bmp],37.462720,0
2,[monarch.bmp],43.264560,0
3,[lighthouse.bmp],19.510097,0
4,[plane.bmp],72.829422,0


In [5]:
# Calculate mse, ssim, lpips and store in the dfnew
loss_fn = lpips.LPIPS(net='alex')

img_path = '/home/sangeeta/Desktop/phd/sem/10th sem/Image_processing/assignment4/ques3/hw5/gblur'
ref_path = '/home/sangeeta/Desktop/phd/sem/10th sem/Image_processing/assignment4/ques3/hw5/refimgs'

dfnew['mse'] = ""
dfnew['ssim'] = ""
dfnew['lpips'] = ""
for i in range(len(dfnew['refnames_blur'])):
    ref_img = cv2.imread(os.path.join(ref_path,dfnew['refnames_blur'][i].item()))
    img = cv2.imread(os.path.join(img_path,'img'+str(i+1)+'.bmp'))
    
    ref_img_ten = torch.tensor(ref_img.transpose([2,0,1])).unsqueeze(0)
    img_ten = torch.tensor(img.transpose([2,0,1])).unsqueeze(0)
    dfnew['mse'][i]  = mean_squared_error(ref_img.reshape([-1]),img.reshape([-1]))
    dfnew['ssim'][i]= ssim(ref_img,img,data_range=img.max() - img.min(),multichannel=True)
    dfnew['lpips'][i] = loss_fn.forward(ref_img_ten,img_ten).item()

dfnew.to_csv("result.csv",index=False)
dfnew.head()

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/sangeeta/miniconda3/envs/py37/lib/python3.7/site-packages/lpips/weights/v0.1/alex.pth


,refnames_blur,blur_dmos,blur_orgs,mse,ssim,lpips
0,[caps.bmp],42.127765,0,18.6194,0.890867,0.125209
1,[churchandcapitol.bmp],37.462720,0,38.2939,0.879016,0.12677
2,[monarch.bmp],43.264560,0,29.898,0.861866,0.205707
3,[lighthouse.bmp],19.510097,0,7.40016,0.989553,0.0169834
4,[plane.bmp],72.829422,0,40.4085,0.690554,0.232754


In [6]:
## Calculate Spearman rank order correlation coefficient for the metrics calculated in Part A

sp_dmos = stats.spearmanr(dfnew['blur_dmos'],dfnew['blur_dmos'])
sp_mse = stats.spearmanr(dfnew["mse"],dfnew['blur_dmos'])
sp_ssim = stats.spearmanr(dfnew["ssim"],dfnew['blur_dmos'])
sp_lpips = stats.spearmanr(dfnew["lpips"],dfnew['blur_dmos'])

In [7]:
print("Spearman rank order correlation coefficient for")
print(f"Human Opinion: {sp_dmos}")
print(f"Mean squared error in pixel domain: {sp_mse}")
print(f"Single scale structural similarity index: {sp_ssim}")
print(f"Learned perceptual image patch similarity metric: {sp_lpips}")

Spearman rank order correlation coefficient for
Human Opinion: SpearmanrResult(correlation=0.9999999999999998, pvalue=0.0)
Mean squared error in pixel domain: SpearmanrResult(correlation=0.6607345299952763, pvalue=1.530215622624923e-19)
Single scale structural similarity index: SpearmanrResult(correlation=-0.9213194772476775, pvalue=1.5504169627935235e-60)
Learned perceptual image patch similarity metric: SpearmanrResult(correlation=0.8724137931034482, pvalue=2.6518716703443e-46)
